<a href="https://colab.research.google.com/github/goldifsh/multiagentes/blob/main/M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install agentpy matplotlib numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.9/778.9 kB 14.0 MB/s eta 0:00:00


In [15]:
import agentpy as ap
import numpy as np
import random
import matplotlib.pyplot as plt

class CleaningAgent(ap.Agent):
    def setup(self):
        self.movements = 0

    def step(self):
        pos = self.model.grid.positions[self]

        if self.model.grid.dirt[pos]:
            self.model.grid.dirt[pos] = False
            self.model.dirty_cells -= 1
            self.model.cleaned_cells += 1
        else:
            directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1),  (1, 0), (1, 1)]
            dr, dc = random.choice(directions)
            new_row = pos[0] + dr
            new_col = pos[1] + dc

            if (0 <= new_row < self.model.p.rows) and (0 <= new_col < self.model.p.cols):
                self.model.grid.move_to(self, (new_row, new_col))
                self.movements += 1
                self.model.total_movements += 1


class CleaningModel(ap.Model):
    def setup(self):
        self.grid = ap.Grid(self, (self.p.rows, self.p.cols), track_empty=True)
        self.total_movements = 0
        self.cleaned_cells = 0

        total_cells = self.p.rows * self.p.cols
        dirty_cells = int(total_cells * self.p.dirty_percent / 100)
        self.grid.dirt = np.zeros((self.p.rows, self.p.cols), dtype=bool)

        dirty_positions = random.sample(list(np.ndindex(self.grid.dirt.shape)), dirty_cells)
        for pos in dirty_positions:
            self.grid.dirt[pos] = True
        self.dirty_cells = dirty_cells

        self.agents = ap.AgentList(self, self.p.num_agents, CleaningAgent)
        self.grid.add_agents(self.agents, positions=[(0, 0)] * self.p.num_agents)

    def step(self):
        self.agents.step()
        if self.dirty_cells == 0:
            self.stop()

    def end(self):
        clean_percent = 100 - (self.dirty_cells / (self.p.rows * self.p.cols) * 100)
        self.report('final_clean_percent', float(clean_percent))
        self.report('total_movements', int(self.total_movements))
        self.report('time_to_clean', int(self.t if self.dirty_cells == 0 else self.p.max_time))

def plot(model, ax):
    grid_data = np.zeros((model.p.rows, model.p.cols))

    for i in range(model.p.rows):
        for j in range(model.p.cols):
            if model.grid.dirt[i, j]:
                grid_data[i, j] = 0.5

    for agent in model.agents:
        r, c = model.grid.positions[agent]
        grid_data[r, c] = 1

    ax.clear()
    ax.imshow(grid_data, cmap="coolwarm", vmin=0, vmax=1)
    ax.set_title(f"Paso {model.t}")
    ax.set_xticks([])
    ax.set_yticks([])

parameters = {
    'rows': 6,
    'cols': 6,
    'num_agents': 3,
    'dirty_percent': 30,
    'steps': 100,
    'max_time': 100,
}

model = CleaningModel(parameters)

fig, ax = plt.subplots(figsize=(5, 5))
animation = ap.animate(model, fig, ax, plot)

from IPython.display import HTML
HTML(animation.to_jshtml())


In [16]:
results = model.run()

time_to_clean = results.reporters['time_to_clean'].iloc[0]
final_clean_percent = results.reporters['final_clean_percent'].iloc[0]
total_movements = results.reporters['total_movements'].iloc[0]

print("Tiempo hasta limpieza completa (o máx):", time_to_clean, "pasos")
print("Porcentaje final de celdas limpias:", f"{final_clean_percent:.2f}%")
print("Número total de movimientos de los agentes:", total_movements)


Completed: 100 steps
Run time: 0:00:00.002385
Simulation finished
Tiempo hasta limpieza completa (o máx): 100 pasos
Porcentaje final de celdas limpias: 88.89%
Número total de movimientos de los agentes: 44
